# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [52]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [53]:
#Import WeatherPy data
cities_df = pd.read_csv("../Weatherpy/weather_check_output.csv")
cities_df


,Unnamed: 0,city,lat,lon,temp_max,humidity,clouds,wind,country,description
0,0,luderitz,-26.65,15.16,56.34,89,1,14.45,NaN,clear sky
1,1,dikson,73.51,80.55,10.60,95,84,17.78,RU,broken clouds
2,2,jamestown,42.10,-79.24,43.00,100,90,2.68,US,overcast clouds
3,3,mataura,-46.19,168.86,57.00,67,75,8.99,NZ,broken clouds
4,4,rikitea,-23.12,-134.97,75.54,86,100,15.08,PF,light rain
...,...,...,...,...,...,...,...,...,...,...
549,549,belfast,54.58,-5.93,43.00,93,100,2.24,GB,overcast clouds
550,550,pisco,-13.70,-76.22,63.00,77,0,6.93,PE,clear sky
551,551,grindavik,63.84,-22.43,37.40,74,75,24.16,IS,broken clouds
552,552,wexford,52.33,-6.46,52.00,93,100,8.01,IE,light rain


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [54]:
#Configure gmaps with APIkey
gmaps.configure(api_key = g_key)

#Make lat and lng columns
latitudes = cities_df["lat"].tolist()
longitudes = cities_df["lon"].tolist()
humidity_list = cities_df["humidity"].tolist()

lat_lng_list = list(zip(latitudes,longitudes))

In [55]:
# Format figure
figure_layout = {
    'width': '400px',
    'height': '300px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'}

fig = gmaps.figure(layout = figure_layout)

#fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(lat_lng_list, weights=humidity_list,
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='300px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [49]:
#My ideal weather condition
ideal_weather = cities_df.loc[(cities_df["wind"]<20) & (cities_df["temp_max"]>70) & (cities_df["temp_max"]<85) & 
                              (cities_df["clouds"] > 0)].dropna()    
ideal_weather

,Unnamed: 0,city,lat,lon,temp_max,humidity,clouds,wind,country,description
4,4,rikitea,-23.12,-134.97,75.54,86,100,15.08,PF,light rain
5,5,garissa,-0.45,39.64,76.08,75,61,11.39,KE,broken clouds
7,7,kirakira,-10.45,161.92,83.43,77,100,7.14,SB,moderate rain
16,16,vaini,-21.20,-175.20,77.00,78,40,9.17,TO,scattered clouds
17,17,airai,-8.93,125.41,78.57,57,63,3.40,TL,light rain
...,...,...,...,...,...,...,...,...,...,...
532,532,nicolas bravo,18.45,-88.92,74.97,94,100,4.81,MX,overcast clouds
538,538,gouyave,12.16,-61.73,78.80,94,75,13.87,GD,thunderstorm
545,545,nishihara,26.18,127.76,82.40,44,75,16.11,JP,broken clouds
548,548,kota kinabalu,5.97,116.07,78.80,83,40,18.34,MY,scattered clouds


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [51]:
hotel_df = ideal_weather
hotel_df

,Unnamed: 0,city,lat,lon,temp_max,humidity,clouds,wind,country,description
4,4,rikitea,-23.12,-134.97,75.54,86,100,15.08,PF,light rain
5,5,garissa,-0.45,39.64,76.08,75,61,11.39,KE,broken clouds
7,7,kirakira,-10.45,161.92,83.43,77,100,7.14,SB,moderate rain
16,16,vaini,-21.20,-175.20,77.00,78,40,9.17,TO,scattered clouds
17,17,airai,-8.93,125.41,78.57,57,63,3.40,TL,light rain
...,...,...,...,...,...,...,...,...,...,...
532,532,nicolas bravo,18.45,-88.92,74.97,94,100,4.81,MX,overcast clouds
538,538,gouyave,12.16,-61.73,78.80,94,75,13.87,GD,thunderstorm
545,545,nishihara,26.18,127.76,82.40,44,75,16.11,JP,broken clouds
548,548,kota kinabalu,5.97,116.07,78.80,83,40,18.34,MY,scattered clouds


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [57]:
# Add marker layer ontop of heat map
locations = [["Lat", "Lng"]]
markers = gmaps.(locations)

# Display figure


NameError: name 'narrowed_city_df' is not defined